## **Named Entity Recognition**

In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', None)

In [31]:
books = pd.read_csv('../Data/childrens_books.csv')

In [32]:
books

,Ranking,Title,Author,Year,Rating,Description
0,1,Where the Wild Things Are,Maurice Sendak,1963,4.25,"Where the Wild Things Are follows Max, a young boy who, after being sent to his room for misbehaving, imagines sailing to an island filled with wild creatures. As their king, Max tames the beasts and eventually returns home to find his supper waiting for him. This iconic book explores themes of imagination, adventure, and the complex emotions of childhood, all captured through Sendak's whimsical illustrations and story."
1,2,The Very Hungry Caterpillar,Eric Carle,1969,4.34,"The Very Hungry Caterpillar tells the story of a caterpillar who eats through a variety of foods before eventually becoming a butterfly. Eric Carle’s use of colorful collage illustrations and rhythmic text has made this book a beloved classic for young readers. The simple, engaging story introduces children to days of the week, counting, and the concept of metamorphosis. It’s a staple in early childhood education."
2,3,The Giving Tree,Shel Silverstein,1964,4.38,"The Giving Tree is a touching and bittersweet story about a tree that gives everything it has to a boy over the course of his life. As the boy grows up, he takes more from the tree, and the tree continues to give, even when it has little left. Silverstein’s minimalist text and illustrations convey deep themes of unconditional love, selflessness, and the passage of time. It has sparked much discussion about relationships and sacrifice."
3,4,Green Eggs and Ham,Dr. Seuss,1960,4.31,"In Green Eggs and Ham, Sam-I-Am tries to convince a reluctant character to try a dish of green eggs and ham, despite his resistance. Through repetition and rhyme, Dr. Seuss’s classic story about being open to new experiences encourages children to be adventurous and try things outside their comfort zone. The playful illustrations and humorous dialogue make it a fun and educational read for young readers."
4,5,Goodnight Moon,Margaret Wise Brown,1947,4.31,"Goodnight Moon is a gentle, rhythmic bedtime story where a little bunny says goodnight to everything in his room, from the moon to the ""quiet old lady whispering hush."" Its repetitive structure and comforting tone make it ideal for young children. The simple illustrations by Clement Hurd complement the soothing nature of the story, making it a beloved classic for sleep-time reading."
...,...,...,...,...,...,...
95,96,Stone Soup,Jon J. Muth,2003,4.18,"Stone Soup is a classic folktale retold by Jon J. Muth, in which three soldiers arrive in a village, claiming they can make soup from stones. The villagers, initially hesitant, eventually contribute ingredients to the pot, and together they create a delicious meal. This story is a heartwarming lesson about sharing, community, and cooperation, enhanced by Muth’s beautiful watercolor illustrations."
96,97,A Light in the Attic,Shel Silverstein,1981,4.36,"A Light in the Attic is a collection of quirky poems and illustrations by Shel Silverstein, exploring a wide range of topics with wit, humor, and imagination. The poems are funny, thoughtful, and sometimes absurd, but they all carry a sense of wonder and creativity. Silverstein’s distinctive voice and whimsical illustrations make this book a timeless collection that continues to entertain children and adults alike."
97,98,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",J.K. Rowling,1999,4.58,"Harry Potter and the Prisoner of Azkaban is the third book in the Harry Potter series, where Harry returns to Hogwarts for his third year and uncovers secrets about his past. With the arrival of the mysterious Sirius Black, Harry must navigate dark truths and face his fears. This thrilling installment explores themes of loyalty, friendship, and identity, marking a turning point in the magical world of Harry Potter."
98,99,"Harry Potter and the Chamber of Secrets (Harry Potter, #2)",J.K. Rowling,1998,4.43,"Harry Potter and the Chamber of Secrets is the second book in the Harry P

In [4]:
## Suppress the warnings
import warnings
warnings.filterwarnings("ignore")

In [33]:
from transformers import pipeline, logging
logging.set_verbosity_error()

In [47]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("ner", 
                model="dbmdz/bert-large-cased-finetuned-conll03-english",
                device='mps',
                aggregation_strategy='SIMPLE'
               )

In [110]:
pipe(books.Description[0])

[{'entity_group': 'MISC',
  'score': 0.94625187,
  'word': 'Where the Wild Things Are',
  'start': 0,
  'end': 25},
 {'entity_group': 'PER',
  'score': 0.9990614,
  'word': 'Max',
  'start': 34,
  'end': 37},
 {'entity_group': 'PER',
  'score': 0.9984414,
  'word': 'Max',
  'start': 175,
  'end': 178},
 {'entity_group': 'PER',
  'score': 0.9789461,
  'word': 'Sendak',
  'start': 380,
  'end': 386}]

In [113]:
[entity['word'] for entity in pipe(books.Description[0]) if entity['entity_group']=='PER']

['Max', 'Max', 'Sendak']

In [114]:
words = books['Description'].apply(
    lambda row: [entity['word'] 
                 for entity in pipe(row) 
                 if entity['entity_group']=='PER']
)

In [116]:
named_entities = list(set(words.explode().dropna().to_list()))

In [117]:
named_entities = [entities for entities in named_entities if '#' not in entities]

In [119]:
len(named_entities)

154

In [125]:
ne_without_authors = [ne for ne in named_entities if ne not in books.Author.to_list()]

In [128]:
len(ne_without_authors)

145